In [ ]:
# source /home/sanjana/ML_Sem3_Project/chess-nlp-env/bin/activate

import chess.pgn  # (Portable Game Notation)
import pandas as pd
from tqdm import tqdm  # For a progress bar, useful in case of large files
import csv
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import torch

# 1) and 2)
# Start by activating the virtual environment: source chess-nlp-env/bin/activate

pgn_path = "../data/processed/lichess_db_standard_rated_2014-09.pgn"
output_csv = "../data/processed/chess_games.csv"

num_games = 10000  # First 5000 games from the dataset
min_plies_required = 6

with open(output_csv, "w", newline="", encoding="utf-8") as fout:  # write mode ('w') with newline='' to prevent extra blank rows.
    writer = csv.writer(fout)  # writer object is responsible for handling the CSV formatting.
    writer.writerow(["white", "white_elo", "black", "black_elo", "result", "date", "moves"])

    with open(pgn_path, encoding="utf-8") as pgn_file:  # with block ensures the file is automatically closed when done (or on error)
        for _ in tqdm(range(num_games)):
            game = chess.pgn.read_game(pgn_file)  # reads the next PGN entry from the open file and returns a Game object
            if game is None:
                break  # end of file
    
            # 'read_game' reads whole PGN blocks (headers + move text)
            try:
                # All the features in the data
                # game.headers: a dict-like mapping of PGN tag names to values
                # .get(key, default): returns value if present, else default '?' prevents KeyError and preserves placeholder if metadata missing.
                white = game.headers.get("White", "?")
                white_elo = game.headers.get("WhiteElo", "?")
                black = game.headers.get("Black", "?")
                black_elo = game.headers.get("BlackElo", "?")
                result = game.headers.get("Result", "?")
                date = game.headers.get("Date", "?")
    
                board = game.board()  # An actual chess board
                moves = []
                for move in game.mainline_moves():  # Gives a move object
                    try:
                        san = board.san(move)  # board.san(move): Python-Chess verifies that the move is legal from the current position.
                        moves.append(san)
                        board.push(move)  # updates board state
                    except Exception:
                        # Skip illegal move: illegal move raises an Assertion error
                        moves = []
                        break
    
                if not moves or len(moves) < min_plies_required:
                    continue  # skip this game and minimum number of plies/tokens for a checkmate is 6: The fool's mate
    
                writer.writerow([white, white_elo, black, black_elo, result, date, " ".join(moves)])  # Instead of storing game as a list, saving memory
                                                                                      # and writing directly to csv

            except Exception as e:
                # Skip the whole game if anything else fails
                continue

print(f"\nFinished processing {num_games} games.")
print(f"Clean data written to: {output_csv}")

data = pd.read_csv(output_csv)
print(f"Number of valid games: {len(data)}.\n")
print(data.head())

data.info()
print("\n---\n")  # separator

print("Data shape:", data.shape, "\n")

# This splits each move by " " and creates a list of all moves in a column, then calculates their length and then this new Series
# that contains lengths of moves in a game is used for getting Statistical Summary of the number of moves in each game
print(data['moves'].apply(lambda s: len(s.split())).describe(), "\n")  # Analyzes the distribution of move lengths

# Adding the num_moves column that will have number of moves in each game
data["moves_list"] = data["moves"].apply(lambda x: x.split(" "))
data["num_moves"] = data["moves_list"].apply(len)

print(data['result'].value_counts())

# Histogram of game lengths

data['num_moves'].hist(bins = 30)
plt.xlabel("Number of Moves")
plt.ylabel("Frequency")
plt.show()



# 3) and 4)

# Building a vocab dictionary

# each row is a list in data['moves_list']
# .explode() expands it so that each list element becomes a separate row and .unique() extracts distinct SAN strings
unique_moves = data['moves_list'].explode().unique()
print(unique_moves[:20])

vocab = {move: idx for idx, move in enumerate(unique_moves, start = 2)}
cnt = 0
print("\nVocab")
for move, idx in vocab.items():
    if cnt < 20:
        print(move, ": ", idx)
        cnt += 1
    else:
        break

vocab['<PAD>'] = 0
vocab['<UNK>'] = 1

inv_vocab = {id: mv for mv, id in vocab.items()}  # Inverse vocab for decoding
cnt = 0
print("\nInverse vocab")
for idx, move in inv_vocab.items():
    if cnt < 20:
        print(idx, ": ", move)
        cnt += 1
    else:
        break

# Now, converting every game’s move list to integer IDs
data["encoded_moves"] = data['moves_list'].apply(lambda moves: [vocab[m] for m in moves])

print("\nEncoded moves list")
data["encoded_moves"].head()

len(unique_moves)

# 5) 
# Creating Input_sequence and Target pairs (X, y)

X = []
y = []

for moves in data["moves_list"]:
    for i in range (1, len(moves)):
        seq_input = moves[:i]  # All moves till i
        seq_output = moves[i]  # Next move
        X.append(seq_input)
        y.append(seq_output)


# Converting input and target to IDs 

PAD = vocab['<PAD>']
UNK = vocab['<UNK>']

X_ids = []
y_ids = []

for seq_input, seq_output in zip(X, y):
    input_ids = [vocab.get(m, UNK) for m in seq_input]  # Unknown moves (not in vocab) are replaced with <UNK> 
    output_ids = vocab.get(seq_output, UNK)

    X_ids.append(input_ids)
    y_ids.append(output_ids)

y_ids[100:110]

# 6)
# Converting X_ids and y_ids to tensors and padding the sequences in X_ids to bring them to equal length sequences

X_tensor = [torch.tensor(seq) for seq in X_ids]  # 2D tensor of size: N x max_move_sequence_length (list of tensors)
y_tensor = torch.tensor(y_ids)  # 1D tensor

X_padded = pad_sequence(X_tensor, batch_first = True, padding_value = PAD)  # batch_first = True: number of samples



# Converting moves to tokens and building vocabulary where vocab = {move: idx} where idx starts at 4 (reserve 0 PAD, 1 UNK, 2 BOS, 3 EOS)
# Saving vocabulary in vocab.json and inv_vocab for decoding

torch.save({
    'vocab': vocab,          # move → ID mapping
    'inv_vocab': inv_vocab,  # ID → move mapping
    'X_padded': X_padded,
    'y_tensor': y_tensor,
    'data': data
}, "../data/processed/preprocessed_data.pt")

checkpoint = torch.load("../data/processed/preprocessed_data.pt", weights_only=False)

vocab = checkpoint['vocab']
inv_vocab = checkpoint['inv_vocab']
X_padded = checkpoint['X_padded']
y_tensor = checkpoint['y_tensor']
data = checkpoint['data']

print("Loaded preprocessed data successfully.")

# Next steps
# Implement and train LSTM baseline; then implement Transformer from scratch.

'''
1) Split data into training and validation
2) Create DataLoader
3) Define your LSTM model
4) Define Loss and Optimizer
5) Train the model
6) Validate
7) Predict and Decode
'''

